In [4]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
area = 'pfdl'

results_folder = 'E:\\projects_q_30_10_2024\\isttc\\results\\monkey\\'
save_folder = results_folder + 'fixation_period_1000ms\\non_binned\\' + area + '\\acf\\'

csv_data_file_pfdl = results_folder + 'data_' + area + '_fixon_1500ms_fixation.csv'
with open(csv_data_file_pfdl, newline='') as f:
    reader = csv.reader(f)
    sua_list_pfdl = list(reader)
    
n_spike_trains_pfdl = len(sua_list_pfdl)
print('N spike_trains in {} fixON {}'.format(area, n_spike_trains_pfdl))

N spike_trains in pfdl fixON 33018


In [8]:
time_series_a = np.asarray(sua_list_pfdl[33][3:]).astype(int)
time_series_b = np.asarray(sua_list_pfdl[34][3:]).astype(int)
print('time_series_a: {} \ntime_series_b: {}'.format(time_series_a, time_series_b))

time_series_a: [  22  172  267  287  353  472  604  652  722  761  779  844  917  966
 1020 1061 1076 1116 1176 1188 1220 1263 1284 1339 1376 1385 1449] 
time_series_b: [ 197  395  433  465  543  614  760  936 1035 1078 1110 1147 1182 1206
 1222 1234 1247 1274 1292 1365 1402 1447 1490]


In [34]:
dt = 25

n_a = len(time_series_a)
n_b = len(time_series_b)

#### sttc

In [28]:
def run_T(spiketrain, N, dt, t_start, t_stop):
    """
    Calculate the proportion of the total recording time 'tiled' by spikes.
    N: number of spikes
    """
    time_A = 2 * N * dt  # maxium possible time

    if N == 1:  # for just one spike in train
        if spiketrain[0] - t_start < dt:
            time_A = time_A - dt + spiketrain[0] - t_start
        elif spiketrain[0] + dt > t_stop:
            time_A = time_A - dt - spiketrain[0] + t_stop

    else:  # if more than one spike in train
        '''
            This part of code speeds up calculation with respect to the original version
        '''
        diff = np.diff(spiketrain)
        idx = np.where(diff<(2*dt))[0]
        Lidx = len(idx)
        time_A = time_A - 2 * Lidx * dt + diff[idx].sum()

        if (spiketrain[0] - t_start) < dt:
            time_A = time_A + spiketrain[0] - dt - t_start

        if (t_stop - spiketrain[N - 1]) < dt:
            time_A = time_A - spiketrain[-1] - dt + t_stop

    T = (time_A / (t_stop - t_start)) #.item()
    
    return time_A, T

In [36]:
def run_P(spiketrain_1, spiketrain_2, N1, N2, dt):
    """
    Check every spike in train 1 to see if there's a spike in train 2
    within dt
    """
    Nab = 0
    j = 0
    for i in range(N1):
        L=0
        while j < N2:  # don't need to search all j each iteration
            if np.abs(spiketrain_1[i] - spiketrain_2[j]) <= dt:
                Nab = Nab + 1
                L+=1
                break
            elif spiketrain_2[j] > spiketrain_1[i]:
                break
            else:
                j = j + 1
    return Nab

In [38]:
t_start = 0
t_stop = 1500

time_a, t_a = run_T(time_series_a, n_a, dt, t_start, t_stop)
print('time_a {}, t_a {}'.format(time_a, t_a))

time_b, t_b = run_T(time_series_b, n_b, dt, t_start, t_stop)
print('time_b {}, t_b {}'.format(time_b, t_b))

p_a_count = run_P(time_series_a, time_series_b, n_a, n_b, dt)
p_a = p_a_count / float(n_a)
print('p_a_count {}, p_a {}'.format(p_a_count, p_a))

p_b_count = run_P(time_series_b, time_series_a, n_b, n_a, dt)
p_b = p_b_count / float(n_b)
print('p_b_count {}, p_b {}'.format(p_b_count, p_b))

index = 0.5 * (p_a - t_b) / (1 - p_a * t_b) + 0.5 * (p_b - t_a) / (1 - p_b * t_a)
print(index)

time_a 1071, t_a 0.714
time_b 837, t_b 0.558
p_a_count 18, p_a 0.6666666666666666
p_b_count 18, p_b 0.782608695652174
0.16426735691803768


In [41]:
t_start = 0
t_stop = 10000

time_a, t_a = run_T(time_series_a, n_a, dt, t_start, t_stop)
print('time_a {}, t_a {}'.format(time_a, t_a))

time_b, t_b = run_T(time_series_b, n_b, dt, t_start, t_stop)
print('time_b {}, t_b {}'.format(time_b, t_b))

p_a_count = run_P(time_series_a, time_series_b, n_a, n_b, dt)
p_a = p_a_count / float(n_a)
print('p_a_count {}, p_a {}'.format(p_a_count, p_a))

p_b_count = run_P(time_series_b, time_series_a, n_b, n_a, dt)
p_b = p_b_count / float(n_b)
print('p_b_count {}, p_b {}'.format(p_b_count, p_b))

index = 0.5 * (p_a - t_b) / (1 - p_a * t_b) + 0.5 * (p_b - t_a) / (1 - p_b * t_a)
print(index)

time_a 1071, t_a 0.1071
time_b 852, t_b 0.0852
p_a_count 18, p_a 0.6666666666666666
p_b_count 18, p_b 0.782608695652174
0.6768954086975421


In [58]:
time_series_a_short = time_series_a[time_series_a < 200]
time_series_a_short = time_series_a_short[time_series_a_short > 100]
print(time_series_a_short)
time_series_b_short = time_series_b[time_series_b < 300]
print(time_series_b_short)

[172]
[197]


In [60]:
t_start = 150
t_stop = 200

n_a = len(time_series_a_short)
n_b = len(time_series_b_short)

time_a, t_a = run_T(time_series_a_short, n_a, dt, t_start, t_stop)
print('time_a {}, t_a {}'.format(time_a, t_a))

time_b, t_b = run_T(time_series_b_short, n_b, dt, t_start, t_stop)
print('time_b {}, t_b {}'.format(time_b, t_b))

p_a_count = run_P(time_series_a_short, time_series_b_short, n_a, n_b, dt)
p_a = p_a_count / float(n_a)
print('p_a_count {}, p_a {}'.format(p_a_count, p_a))

p_b_count = run_P(time_series_b_short, time_series_a_short, n_b, n_a, dt)
p_b = p_b_count / float(n_b)
print('p_b_count {}, p_b {}'.format(p_b_count, p_b))

index = 0.5 * (p_a - t_b) / (1 - p_a * t_b) + 0.5 * (p_b - t_a) / (1 - p_b * t_a)
print(index)

time_a 47, t_a 0.94
time_b 28, t_b 0.56
p_a_count 1, p_a 1.0
p_b_count 1, p_b 1.0
1.0
